# **Chapter 23: AI/ML System Design**

Artificial Intelligence and Machine Learning have moved from research labs to production systems serving billions of requests daily. Designing ML systems presents unique challenges: models are compute-intensive, require specialized hardware (GPUs/TPUs), need constant retraining as data drifts, and must serve predictions with strict latency requirements. This chapter covers the architecture patterns that power recommendation engines, fraud detection systems, and Large Language Models (LLMs) at scale.

---

## **23.1 Model Serving Architectures**

When deploying machine learning models, the serving strategy determines your latency, throughput, and cost. The choice between batch and real-time inference depends on your use case's tolerance for delay.

### **Batch Inference: Process Everything at Once**

**Concept**: Collect data over a time window (hourly, daily), run inference on all data simultaneously, store results for later retrieval.

**Architecture**:
```
Data Lake (S3/HDFS) → Spark/Flink Job → Model Inference → Feature Store/Database
     ↑                                                      ↓
   Raw Events                                         Pre-computed Predictions
                                                        (User recommendations,
                                                         fraud scores, etc.)
```

**When to Use**:
- Recommendations that update hourly (not instant)
- Overnight fraud risk scoring
- Customer churn prediction (doesn't need to be real-time)
- Training data generation

**Example: Netflix Movie Recommendations**
```
Every 4 hours:
1. Collect viewing history for all 230M users
2. Run collaborative filtering model on GPU cluster
3. Generate "Top 10 for You" list for each user
4. Store in Redis (serves in <1ms when user opens app)

Latency: 4 hours stale, but retrieval is instant
```

**Code Example** (Apache Spark with MLlib):
```python
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALSModel

# Load pre-trained model
model = ALSModel.load("s3://models/collaborative-filtering/v2.1/")

# Batch process all users
user_features = spark.read.parquet("s3://features/user-profiles/")
predictions = model.recommendForUserSubset(user_features, 10)

# Write results to serving layer
predictions.write \
    .format("redis") \
    .option("table", "recommendations") \
    .mode("overwrite") \
    .save()

# 230M users processed in 30 minutes on 100 GPU instances
```

**Advantages**:
- **Cost efficient**: GPUs utilized at 100% (no idle time)
- **High throughput**: Process millions of records in parallel
- **Complex models**: Can use larger models that would be too slow for real-time

**Disadvantages**:
- **Stale predictions**: Results are hours old
- **Cold start**: New users wait for next batch cycle
- **Storage cost**: Must store predictions for all users

### **Real-Time (Online) Inference: Predict on Demand**

**Concept**: Receive request → Extract features → Run model → Return prediction in milliseconds.

**Architecture**:
```
User Request → API Gateway → Feature Store (fetch) → Model Server (GPU) → Response
                   ↓                ↓                      ↓
            Rate Limiting      Pre-computed         Model Inference
                               features            (TensorFlow/PyTorch)
```

**When to Use**:
- Fraud detection (must check transaction now)
- Search ranking (results needed immediately)
- Autocomplete (every keystroke)
- Self-driving cars (real-time object detection)

**Example: Credit Card Fraud Detection**
```
User swipes card → Transaction sent to API → 
    ├─ Fetch user features (avg transaction amount, location history)
    ├─ Run XGBoost model on GPU (2ms inference time)
    ├─ Return "approve" or "decline"
Total latency: 50ms (user doesn't notice delay)
```

**Model Serving Frameworks**:

**TensorFlow Serving**:
```python
# Model server configuration
model_config {
  name: 'fraud_detection'
  base_path: '/models/fraud'
  model_version_policy {
    specific {
      versions: 1
      versions: 2  # Keep both versions for A/B testing
    }
  }
  version_labels {
    key: 'stable'
    value: 1
  }
  version_labels {
    key: 'canary'
    value: 2
  }
}

# Client request
import grpc
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('localhost:8500')
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

request = predict_pb2.PredictRequest()
request.model_spec.name = 'fraud_detection'
request.model_spec.signature_name = 'serving_default'
request.inputs['transaction'].CopyFrom(tf.make_tensor_proto(transaction_data))

result = stub.Predict(request, 10.0)  # 10 second timeout
fraud_score = result.outputs['score'].float_val[0]
```

**TorchServe** (PyTorch):
```python
# handler.py
from ts.torch_handler.base_handler import BaseHandler

class FraudHandler(BaseHandler):
    def preprocess(self, data):
        # Extract features from request
        return torch.tensor([data[0]['body']])
    
    def inference(self, inputs):
        # Run on GPU if available
        with torch.no_grad():
            return self.model(inputs).sigmoid()
    
    def postprocess(self, inference_output):
        return [{"fraud_probability": float(inference_output[0][0])}]

# Deployment
# torch-model-archiver --model-name fraud --version 1.0 --handler handler.py
# torchserve --start --model-store model_store --models fraud=fraud.mar
```

### **Hybrid Architecture: The Lambda Pattern for ML**

Most production systems use both approaches simultaneously:

```
Real-time Path (Latency-critical):
User Search Query → Feature Store → Small Model (BERT-tiny) → Initial Ranking (100ms)
                                                          ↓
Batch Path (Quality-critical):                          Merge
Overnight Spark Job → Large Model (GPT-4) → Re-ranking Scores (stored in cache)
                                                          ↓
                                                    Final Results
```

**Example: Amazon Search**
1. **Real-time**: User types "wireless headphones" → Quick retrieval of 1000 candidates using inverted index → Light model scores them in 50ms
2. **Batch**: Every 6 hours, deep learning model re-ranks all products based on inventory, seasonality, profit margins
3. **Merge**: Combine real-time relevance with batch quality scores

---

## **23.2 Feature Stores: The Feature Management Layer**

A feature store is a centralized repository for storing, serving, and managing machine learning features. It solves the "training-serving skew" problem where models perform differently in training versus production due to inconsistent feature computation.

### **The Problem: Training-Serving Skew**

```python
# Training code (Python/Pandas)
def compute_user_features(df):
    df['avg_purchase_7d'] = df.groupby('user_id')['amount'].rolling(7).mean()
    return df

# Serving code (Java/Production)
public double getAvgPurchase(User user) {
    // Different implementation!
    return database.average(user.getPurchases(), Days.SEVEN);
}

# Result: Model expects normalized values, serving code returns raw values
# Model performance drops from 95% to 70% accuracy in production!
```

### **Feature Store Architecture**

```
┌─────────────────────────────────────────────────────────────┐
│                      Feature Store                           │
├─────────────────────────┬───────────────────────────────────┤
│   Offline Store         │      Online Store                 │
│   (Training Data)       │      (Real-time Serving)          │
│                         │                                   │
│   ┌──────────────┐     │      ┌──────────────┐             │
│   │ Data Lake    │     │      │ Redis/Low    │             │
│   │ (Parquet/    │     │      │ Latency DB   │             │
│   │  Delta Lake) │     │      │              │             │
│   └──────────────┘     │      └──────────────┘             │
│          ↑             │             ↑                      │
│   Feature Pipeline     │      Feature Retrieval API         │
│   (Spark/Flink)        │      (GRPC/REST)                   │
└─────────────────────────┴───────────────────────────────────┘
                    ↓
            Feature Registry (Metadata, Lineage)
```

### **Feast (Open Source Feature Store)**

**Defining Features**:
```python
from feast import Entity, Feature, FeatureView, ValueType
from feast.types import Float32, Int64
from feast import FileSource, RedisSource

# Define the entity (what you're predicting for)
user = Entity(
    name="user_id",
    value_type=ValueType.INT64,
    description="User identifier"
)

# Define data sources
user_transactions = FileSource(
    path="s3://features/user_transactions.parquet",
    event_timestamp_column="timestamp"
)

# Define feature view (how to compute features)
user_stats_view = FeatureView(
    name="user_transaction_stats",
    entities=["user_id"],
    ttl=timedelta(hours=24),  # Feature freshness
    features=[
        Feature(name="avg_transaction_7d", dtype=Float32),
        Feature(name="transaction_count_30d", dtype=Int64),
        Feature(name="preferred_category", dtype=String)
    ],
    online=True,  # Store in Redis for real-time serving
    source=user_transactions
)
```

**Training (Offline)**:
```python
from feast import FeatureStore

store = FeatureStore(repo_path=".")

# Get features for training (batch retrieval from S3)
training_df = store.get_historical_features(
    entity_df=user_ids_with_labels,
    features=[
        "user_transaction_stats:avg_transaction_7d",
        "user_transaction_stats:transaction_count_30d"
    ]
).to_df()

# train_model(training_df) - Features match production exactly
```

**Serving (Online)**:
```python
# Real-time feature retrieval (from Redis, <5ms)
features = store.get_online_features(
    features=[
        "user_transaction_stats:avg_transaction_7d",
        "user_transaction_stats:preferred_category"
    ],
    entity_rows=[{"user_id": 12345}]
).to_dict()

# Pass to model for inference
prediction = model.predict(features)
```

### **Feature Engineering at Scale**

**Stream Processing for Real-Time Features**:
```python
# Apache Flink: Compute real-time features as events arrive
class TransactionFeatureJob:
    def flat_map(self, transaction):
        user_id = transaction.user_id
        amount = transaction.amount
        
        # Maintain running average using stateful operators
        current_avg = self.state.value()
        new_avg = (current_avg * self.state.count + amount) / (self.state.count + 1)
        self.state.update(new_avg)
        
        # Emit feature update
        yield FeatureRow(
            entity_key=user_id,
            feature_name="avg_transaction_realtime",
            value=new_avg,
            timestamp=transaction.timestamp
        )

# Updates feature store in real-time as transactions happen
```

**Feature Validation**:
```python
# Ensure feature quality before serving
from great_expectations import validate

expectations = {
    "avg_transaction_7d": {
        "min_value": 0,
        "max_value": 100000,
        "null_rate": "< 0.01"
    }
}

# Validate features before writing to store
if not validate(computed_features, expectations):
    alert_data_team("Feature validation failed!")
    # Don't pollute feature store with bad data
```

---

## **23.3 Model Versioning and A/B Testing**

Machine learning models are software artifacts that need versioning, testing, and gradual rollouts—just like regular code, but with additional complexity around data dependencies and performance drift.

### **Model Registry (MLflow Example)**

```python
import mlflow
import mlflow.sklearn

with mlflow.start_run():
    # Log parameters
    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", 10)
    
    # Log metrics
    mlflow.log_metric("accuracy", 0.95)
    mlflow.log_metric("f1_score", 0.94)
    mlflow.log_metric("inference_latency_ms", 12)
    
    # Log model artifact
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        registered_model_name="fraud-detection-model"
    )
    
    # Log feature dependencies
    mlflow.log_artifact("features.yaml")  # Schema of expected inputs

# Model versions: v1.0 (stable), v1.1 (candidate), v2.0 (experimental)
```

### **Shadow Deployment: Testing Without Risk**

Before serving a new model to users, run it in "shadow mode"—process real requests but don't return results to users. Compare shadow model predictions against production model.

```
User Request → Load Balancer → Production Model (v1.0) → Response to User
                    ↓
               Shadow Model (v2.0) → Results logged to analytics
                    ↓
               Compare: v1.0 predicted 0.9 fraud, v2.0 predicted 0.3
               If disagreement > threshold, alert data scientists
```

**Implementation**:
```python
class ShadowModelMiddleware:
    def __init__(self):
        self.production_model = load_model("v1.0")
        self.shadow_model = load_model("v2.0")
        self.comparison_queue = []
    
    def predict(self, request):
        # Production path (blocking, must be fast)
        prod_result = self.production_model.predict(request)
        
        # Shadow path (async, non-blocking)
        def shadow_predict():
            shadow_result = self.shadow_model.predict(request)
            self.log_comparison(request, prod_result, shadow_result)
        
        threading.Thread(target=shadow_predict).start()
        
        return prod_result  # User only sees production result
```

### **A/B Testing for ML Models**

**Traffic Splitting Strategies**:

**1. Random Split (User-level)**:
```python
def route_request(user_id):
    # Consistent hashing ensures same user always hits same model
    bucket = hash(user_id) % 100
    
    if bucket < 50:
        return model_v1  # 50% traffic
    else:
        return model_v2  # 50% traffic
```

**2. Canary Deployment**:
```python
def route_request(user_id, context):
    # Start with 1% of traffic
    if hash(user_id) % 100 == 0:
        return model_v2
    
    # Monitor error rates, latency for 1 hour
    # If healthy, increase to 10%, then 50%, then 100%
```

**3. Multi-Armed Bandit (Automatic Optimization)**:
```python
# Epsilon-greedy strategy: 90% exploit best model, 10% explore alternatives
if random.random() < 0.1:
    # Exploration: Try random model to gather data
    model = random.choice([model_a, model_b, model_c])
else:
    # Exploitation: Use model with highest conversion rate
    model = get_best_performing_model()

# Automatically shifts traffic to best model based on real-time metrics
```

**Monitoring Model Performance**:
```python
# Track model drift over time
class ModelMonitor:
    def check_drift(self, recent_predictions, baseline_distribution):
        # Statistical test (Kolmogorov-Smirnov) for distribution shift
        drift_score = ks_test(recent_predictions, baseline_distribution)
        
        if drift_score > 0.1:
            alert("Model drift detected! Retraining required.")
            # Trigger automated retraining pipeline
            self.trigger_retraining()
    
    def check_latency(self, response_times):
        p99_latency = np.percentile(response_times, 99)
        if p99_latency > 100:  # SLA: 100ms
            alert("Latency degradation detected")
            # Roll back to previous version or scale up infrastructure
```

---

## **23.4 Vector Databases: Semantic Search at Scale**

Traditional databases search by exact match (SQL `WHERE` clauses) or inverted indexes (Elasticsearch). Vector databases enable "semantic search"—finding similar items based on meaning rather than keywords.

### **Embeddings: Converting Data to Vectors**

An embedding is a numerical representation (vector) of data (text, images, audio) where "similar" items are close together in vector space.

```
Text: "king" → [0.2, -0.5, 0.8, ..., 0.1] (768 dimensions)
Text: "queen" → [0.22, -0.48, 0.79, ..., 0.12] (close to king)
Text: "apple" → [-0.9, 0.3, -0.2, ..., 0.8] (far from king)
```

**Generating Embeddings** (OpenAI API):
```python
import openai

def get_embedding(text):
    response = openai.Embedding.create(
        input=text,
        model="text-embedding-3-large"
    )
    return response['data'][0]['embedding']  # 3072-dimensional vector

# Example
embedding = get_embedding("machine learning system design")
# Returns: [0.012, -0.034, 0.256, ...] (3072 numbers)
```

### **Approximate Nearest Neighbor (ANN) Search**

Finding the exact closest vectors in high-dimensional space is slow (O(n)). ANN algorithms trade 1-2% accuracy for 1000x speedup.

**HNSW (Hierarchical Navigable Small World)** - Used by Pinecone, Weaviate:
```
Layer 2 (Sparse):        A ────────→ D
                              ↘
Layer 1 (Medium):      A → B → C → D → E
                          ↘   ↗
Layer 0 (Dense):     A-B-C-D-E-F-G-H-I-J (all connections)

Search: Start at top layer, greedily navigate toward query
       When no closer nodes, drop to next layer
       Repeat until layer 0 (exact local search)
       
Complexity: O(log n) instead of O(n)
```

### **Pinecone (Managed Vector DB)**

```python
import pinecone

# Initialize
pinecone.init(api_key="key", environment="us-west1-gcp")
index = pinecone.Index("product-catalog")

# Upsert vectors (product descriptions → embeddings)
vectors = [
    ("prod_1", [0.1, -0.2, ...], {"category": "electronics", "price": 299}),
    ("prod_2", [0.15, -0.18, ...], {"category": "electronics", "price": 499}),
    # ... millions of products
]
index.upsert(vectors=vectors)

# Query: Find similar products
query_embedding = get_embedding("wireless noise canceling headphones")
results = index.query(
    vector=query_embedding,
    top_k=10,
    filter={"category": {"$eq": "electronics"}, "price": {"$lt": 400}}
)

# Returns: prod_1 (score: 0.95), prod_5 (score: 0.92), etc.
```

### **pgvector (PostgreSQL Extension)**

For applications already using PostgreSQL, pgvector adds vector capabilities without new infrastructure.

```sql
-- Enable extension
CREATE EXTENSION vector;

-- Create table with vector column
CREATE TABLE products (
    id SERIAL PRIMARY KEY,
    name TEXT,
    embedding vector(768)  -- 768 dimensions
);

-- Create IVFFlat index for fast ANN search
CREATE INDEX ON products USING ivfflat (embedding vector_cosine_ops)
WITH (lists = 100);  -- Number of clusters (tune based on data size)

-- Insert data
INSERT INTO products (name, embedding) 
VALUES ('iPhone 15', '[0.1, -0.2, 0.3, ...]');

-- Semantic search (find 5 most similar products)
SELECT name, 1 - (embedding <=> '[0.1, -0.15, ...]') AS similarity
FROM products
WHERE 1 - (embedding <=> '[0.1, -0.15, ...]') > 0.8  -- Similarity threshold
ORDER BY embedding <=> '[0.1, -0.15, ...]'
LIMIT 5;
```

### **Vector Database Comparison**

```
Database     Best For                Latency      Scale
─────────────────────────────────────────────────────────
Pinecone     Production ML apps      <10ms        Billions
Weaviate     Semantic search         <20ms        Millions
Milvus       Open-source large scale <50ms        10B+
pgvector     Existing Postgres users <100ms       Millions
Redis        Real-time caching       <5ms         Millions
```

---

## **23.5 LLM Integration: RAG Architecture and Prompt Engineering at Scale**

Large Language Models (GPT-4, Claude, Llama) require special architectural patterns due to their size (hundreds of gigabytes), cost (per-token pricing), and latency (1-10 seconds).

### **Retrieval-Augmented Generation (RAG)**

LLMs have knowledge cutoffs and hallucinate facts. RAG grounds LLM responses in your private data.

**Architecture**:
```
User Query → Embedding Model → Vector DB (your docs) → Top-K Chunks → 
                                                      ↓
Prompt: "Context: [Chunks] Question: [Query]" → LLM API → Response
```

**Example: Customer Support Chatbot**
```
User: "What's your return policy for electronics?"

1. Convert query to vector: "return policy electronics" → [0.2, -0.1, ...]
2. Search vector DB: Find return_policy.pdf chunks 3, 7, 12
3. Retrieve text chunks:
   - "Electronics can be returned within 30 days..."
   - "Items must be in original packaging..."
   - "Refunds processed in 5-7 business days..."
4. Construct prompt:
   "Context: Electronics can be returned within 30 days...
    Question: What's your return policy for electronics?
    Answer:"
5. Send to LLM (GPT-4)
6. Return: "You can return electronics within 30 days in original packaging..."
```

**Implementation**:
```python
class RAGPipeline:
    def __init__(self):
        self.vector_store = pinecone.Index("company-docs")
        self.llm = OpenAIClient(model="gpt-4")
    
    def answer_query(self, user_query):
        # Step 1: Retrieve relevant documents
        query_embedding = self.embed(user_query)
        matches = self.vector_store.query(query_embedding, top_k=5)
        
        # Step 2: Build context
        context = "\n\n".join([match.text for match in matches])
        
        # Step 3: Construct prompt with context
        prompt = f"""Answer the question based on the following context.
If the answer is not in the context, say "I don't know".

Context:
{context}

Question: {user_query}

Answer:"""

        # Step 4: Generate response
        response = self.llm.complete(prompt, max_tokens=500)
        
        return response
```

### **Prompt Engineering at Scale**

**Prompt Versioning and Management**:
```python
# Don't hardcode prompts in code!
# Use a prompt registry (like PromptLayer or simple config)

# prompts.yaml
prompts:
  customer_support:
    version: "2.3"
    template: |
      You are a helpful support agent for {company_name}.
      Context: {context}
      Customer question: {question}
      Provide a concise, friendly answer.
    parameters:
      temperature: 0.7
      max_tokens: 300

# Code
from jinja2 import Template

def load_prompt(prompt_id, variables):
    prompt_config = registry.get(prompt_id)
    template = Template(prompt_config.template)
    return template.render(**variables)

# A/B test different prompts
if user_bucket == "A":
    prompt = load_prompt("customer_support_v2.3", vars)
else:
    prompt = load_prompt("customer_support_v2.4_experimental", vars)
```

**Caching LLM Responses**:
```python
class LLMCache:
    def __init__(self):
        self.cache = RedisCache()
        self.semantic_cache = VectorStore()  # For similar queries
    
    def get(self, query):
        # Exact match cache (cheap, fast)
        if cached := self.cache.get(hash(query)):
            return cached
        
        # Semantic cache (expensive, but catches paraphrases)
        query_embedding = embed(query)
        similar = self.semantic_cache.similarity_search(query_embedding, threshold=0.95)
        if similar:
            return similar[0].response  # Return cached response for similar query
        
        return None

# Cache hit saves $0.01-0.10 per request (important at scale!)
# Also reduces latency from 2s to 10ms
```

### **Handling LLM Constraints**

**Rate Limiting and Token Management**:
```python
class LLMRateLimiter:
    def __init__(self):
        # OpenAI limits: 10,000 requests/minute, 2M tokens/minute
        self.request_bucket = TokenBucket(capacity=10000, refill_rate=10000/60)
        self.token_bucket = TokenBucket(capacity=2000000, refill_rate=2000000/60)
    
    def call(self, prompt):
        tokens = count_tokens(prompt)
        
        if not self.request_bucket.consume(1):
            raise RateLimitError("Too many requests")
        
        if not self.token_bucket.consume(tokens):
            raise RateLimitError("Token quota exceeded")
        
        return openai.Completion.create(prompt=prompt)

# Queue-based architecture for burst handling
class LLMQueue:
    def __init__(self):
        self.queue = PriorityQueue()
        self.workers = [Thread(target=self.process) for _ in range(10)]
    
    def enqueue(self, request, priority=1):
        # Priority: 0 = urgent, 1 = normal, 2 = batch
        self.queue.put((priority, time.time(), request))
    
    def process(self):
        while True:
            priority, timestamp, request = self.queue.get()
            
            # Exponential backoff for rate limits
            for attempt in range(5):
                try:
                    response = llm.call(request.prompt)
                    request.callback(response)
                    break
                except RateLimitError:
                    time.sleep(2 ** attempt)  # 1s, 2s, 4s, 8s, 16s
```

**Model Routing (Route to Cheapest/Fastest Model)**:
```python
def route_query(user_query):
    complexity = classify_complexity(user_query)
    
    if complexity == "simple":
        # Use GPT-3.5 ($0.002/1K tokens, 100ms latency)
        return "gpt-3.5-turbo"
    elif complexity == "complex":
        # Use GPT-4 ($0.06/1K tokens, 2s latency)
        return "gpt-4"
    elif complexity == "coding":
        # Use Claude for code (better at programming)
        return "claude-3-opus"
    
    # Default to middle tier
    return "gpt-4-turbo-preview"
```

---

## **23.6 Key Takeaways**

1. **Choose batch for throughput, real-time for latency**: Batch inference is 10x cheaper but hours stale; online inference provides instant results at higher cost.

2. **Feature stores eliminate training-serving skew**: Centralized feature computation ensures models see consistent data in training and production.

3. **Shadow deployment validates models safely**: Test new models on real traffic without user impact before A/B testing.

4. **Vector databases enable semantic search**: ANN algorithms (HNSW) make billion-scale similarity search feasible in milliseconds.

5. **RAG grounds LLMs in facts**: Retrieve relevant context from vector DB to prevent hallucinations and provide up-to-date answers.

6. **LLMs require special infrastructure**: Rate limiting, semantic caching, and prompt versioning are essential for production LLM applications.

---

## **Chapter Summary**

This chapter covered the unique challenges of ML system design. We explored batch versus real-time inference architectures, implemented feature stores with Feast to ensure consistency, and learned to version and A/B test models safely. We examined vector databases for semantic search and built RAG pipelines to ground LLMs in proprietary data.

The key insight: ML systems are data systems first. The sophistication of your model matters less than the quality, freshness, and consistency of your features. Invest in data infrastructure (feature stores, monitoring, validation) before chasing marginal model improvements.

**Coming up next**: In Chapter 24, we'll explore Edge Computing and IoT—processing data on devices, handling intermittent connectivity, and architecting for the billions of smart devices at the network edge.

---

## **Exercises**

1. **Model Serving Cost Analysis**: Compare costs for serving 10 million predictions daily:
   - Option A: Batch processing on GPU spot instances ($0.50/hour, process nightly)
   - Option B: Real-time inference on dedicated GPUs ($2.00/hour, always on)
   - Option C: Serverless (AWS SageMaker, $0.0001 per inference)
   Calculate monthly costs and identify when each option makes sense.

2. **Feature Store Design**: Design a feature store schema for a ride-sharing app. Identify 5 real-time features (update within seconds) and 5 batch features (update hourly). Write the Feast feature definitions.

3. **Vector Search Implementation**: Implement a semantic search for customer support tickets using pgvector:
   - Create table with vector column (384 dimensions using all-MiniLM-L6-v2 model)
   - Insert 1000 sample tickets with embeddings
   - Write query to find top 5 similar tickets for a new incoming ticket
   - Add metadata filtering (by product category)

4. **RAG Pipeline**: Build a RAG system that:
   - Loads a PDF document, chunks it into 500-token segments
   - Stores chunks in Pinecone with embeddings
   - Answers user questions using GPT-4 with retrieved context
   - Implements semantic caching to avoid repeated LLM calls for similar questions

5. **LLM Rate Limiting**: Design a token bucket algorithm for OpenAI API limits (10,000 req/min, 2M tokens/min). Handle burst traffic gracefully using a queue with exponential backoff on rate limit errors.

---
